# First Assignment (Chapter 2)

In [19]:
# imports
import numpy as np
from importlib import reload

import unyt

from oceanpy import constants
from oceanpy.forces import coriolis

from os_classes.ocea257 import chapter1
from os_classes import plotting as os_plotting


from bokeh import plotting
from bokeh import models

# Analytic

## 2-1 <font color='blue'>(PS1-1)</font> 

### Jupiter

Radius:  $R_J = C_J / 2 \pi$

In [8]:
circum_eq = 448600. * unyt.km # 
R_J = circum / 2 / np.pi  # Radius

### Omega

$\Omega = 2 \pi / t$

In [13]:
t_J = 9.9 * unyt.hr  # 1 day, i.e. full revolution
#
omega_J = 2 * np.pi / t_J
omega_J.to('1/s')

unyt_quantity(0.0001763, '1/s')

### Centrifugal acceleration

$a = \Omega^2 r$

In [12]:
a_centrifugal = omega_J**2 * R_J
a_centrifugal.to('m/s**2')

unyt_quantity(2.21903293, 'm/s**2')

### True acceleration

In [14]:
g_J_measured = 26.4 * unyt.m / unyt.s**2

In [15]:
g_J_true = g_J_measured + a_centrifugal
g_J_true

unyt_quantity(28.61903293, 'm/s**2')

----

## 2-2 Shinkasen  <font color='red'>(not assigned)</font> 

In [17]:
phi = 35 * unyt.deg

### Estimate the Coriolis acceleration 

#### Assume the train motion is purely horizontal

$\frac{dv}{dt} = -f u$

$f = 2 \Omega \sin\phi$

In [18]:
u_train = 185 * unyt.km / unyt.hr

In [22]:
f = coriolis.coriolis_parameter(phi)
f.to('1/s')

unyt_quantity(8.36518044e-05, '1/s')

In [26]:
# Acceleration
dvdt = -f * u_train
dvdt.to('m/s**2')

unyt_quantity(-0.00429877, 'm/s**2')

### This is $\approx 5 \times 10^{-4}$ of the gravitational acceleration

In [27]:
3*unyt.m * 5e-4

unyt_quantity(0.0015, 'm')

### For a track 3m wide, this implies a 1mm deflection.  Probably ignorable.

----

## 2-3 Cannonball <font color='blue'>(PS1-2)</font> 

In [38]:
phi_London = (51. + 31./60)*unyt.deg
Dt = 25 * unyt.s
u_canonball = 120 * unyt.m / unyt.s

### Deflection $\Delta y$

#### Coriolis

$\frac{dv}{dt} = -f u$

$f = 2 \Omega \sin\phi$

In [32]:
f_London = coriolis.coriolis_parameter(phi_London)
f_London.to('1/s')

unyt_quantity(0.00011416, '1/s')

In [36]:
dvdt_London = -f_London * u_canonball
dvdt_London.to('m/s**2')

unyt_quantity(-0.01369967, 'm/s**2')

In [40]:
v_L_approx = dvdt_London * Dt
v_L_approx.to('m/s')

unyt_quantity(-0.34249173, 'm/s')

### Because $v \ll u$, we will ignore 2nd order effects during the $\Delta t = 25$s and take $dv/dt$ as constant.


#### Therefore, integrating and noting $v=0$ at $t=0$, we have

$\Delta y = \frac{1}{2} \frac{dv}{dt} t^2$

In [41]:
Dy = 0.5 * dvdt_London * Dt**2
Dy.to('m')

unyt_quantity(-4.28114668, 'm')

### The cannonball is deflected 4m to the right, i.e. S (assuming it is headed E)

### Other locations

In [43]:
def deflect(phi, Dt = 25 * unyt.s, u_canonball = 120 * unyt.m / unyt.s):
    # Coriolis parameter
    f = coriolis.coriolis_parameter(phi)
    # dv/dt
    dvdt = -f * u_canonball
    # Integrate
    Dy = 0.5 * dvdt * Dt**2
    # Return
    return Dy

### Murmansk

In [44]:
phi_murmansk = (68. + 52./60)*unyt.deg
Dy_murmansk = deflect(phi_murmansk)
Dy_murmansk

unyt_quantity(-5.10126301, 'm')

### Nairobi

In [45]:
phi_nairobi = -(1. + 18./60)*unyt.deg
Dy_nairobi = deflect(phi_nairobi)
Dy_nairobi

unyt_quantity(0.12407913, 'm')

### Any phi

In [50]:
phis = np.linspace(-90, 90, 1000)*unyt.deg

In [51]:
all_deflect = deflect(phis)

In [53]:
# Plot
plotting.output_notebook()

p = plotting.figure(title='Cannonball', x_axis_label='phi (deg)',
                    #x_axis_type="log", y_axis_type="log",
                   y_axis_label='Deflection (m)')#, x_range=(10**-4, 1.),
                   #y_range=(1e-12, 1))

# Plot em
p.line(phis, all_deflect, legend='Cannonball', line_width=2)
#p.scatter(kdx_vals, error_fb, legend='fb', line_width=2, color='red')
#p.scatter(kdx_vals, error_sc, legend='sc', line_width=2, color='green')
#p.scatter(kdx_vals, error_4, legend='4', line_width=2, color='black')


p.legend.location = "top_right"

os_plotting.bokeh_set_fontsize(p, 16)

plotting.show(p)

Loading BokehJS ...

----

## <font color='blue'>PS 1-3</font> -- Gust of wind

In [55]:
phi_gust = 60.*unyt.deg
u = 0.25 * unyt.m / unyt.s

### Following section 2.3 of Cushman-Roisin, the general solution for force-free motion gives:

### $u = V \sin(ft + \phi), \;\; v = V \cos(ft + \phi)$

### with $f$ the Coriolis parameter.  And integrating these velocities gives positions:

### $x = x_0 - \frac{V}{f} \cos (ft + \phi)$

### $y = y_0 + \frac{V}{f} \sin (ft + \phi)$

### Inspection of the equations reveals the interial motion has period

### $T_p = \frac{2 \pi}{f}$

### and radius $R = \frac{V}{f}$

### Computing..

In [57]:
f_gust = coriolis.coriolis_parameter(phi_gust)
f_gust.to('1/s')

unyt_quantity(0.0001263, '1/s')

In [59]:
# Period
Tp = 2 * np.pi / f_gust
Tp.to('hr')

unyt_quantity(13.81855794, 'hr')

In [61]:
# Radius
R_gust = u / f
# Diameter
D_gust = 2 * R_gust
D_gust.to('km')

unyt_quantity(4.37966774, 'km')

### How about $\phi = 5^\circ$S?

In [63]:
f_5S = coriolis.coriolis_parameter(-5.*unyt.deg)

In [66]:
# New period
Tp_5S = Tp * f_gust / np.abs(f_5S)
Tp_5S.to('hr')

unyt_quantity(137.30847612, 'hr')

In [67]:
# New Diameter
D_5S = D_gust * f_gust / np.abs(f_5S)
D_5S.to('km')

unyt_quantity(43.51868735, 'km')

### Both the period and diameter scale as $f^{-1}$. Given that $f \propto \sin\phi$ is smaller at the equator, the period and diameter are both larger at lower latitudes.

----

## <font color='blue'>PS 1-4</font> -- Tank

### Setup

In [87]:
# Tank
D_tank = 30 * unyt.cm
R_tank = D_tank / 2
H_water = 20 * unyt.cm   # Height of water before spinning
# Rotation
T_tank = 1*unyt.minute / 30. # Period
Omega_tank = 2 * np.pi / T_tank
Omega_tank.to('1/s')
# Gravity
g = 9.8 * unyt.m / unyt.s**2

### As the tank is spun up, the water relaxes to a parabolic surface that is an equipotential between Gravity and the centrifugal force.

#### Gravitaional potential in a 'constant' field:

$\Phi_g = g h$ with $h$ the vertical displacement.

#### Potential related to the centrifugal acceleration

$\Phi_c = \int \Omega r^2 dr = \frac{1}{2} \Omega^2 r^2$

#### Equating and solving for $h$:

$h = \frac{1}{2} \frac{\Omega^2}{g} r^2$

### As anticipated, the profile is a parabola

In [72]:
def h_of_r(r):
    h = 0.5 * Omega_tank**2 / g * r**2
    return h

### Evaluated at the edge, $R_{\rm tank} = 15$ cm

In [73]:
h_of_r(R_tank)

unyt_quantity(1.1329903, 'cm')

### Let's plot

In [78]:
Rvals = np.linspace(0., R_tank, 100)
hvals = h_of_r(Rvals)

In [86]:
# Plot
plotting.output_notebook()

p = plotting.figure(title='Rotating tank', x_axis_label='Distance from the center (cm)',
                    #x_axis_type="log", y_axis_type="log",
                   y_axis_label='Relative height (cm)')#, x_range=(10**-4, 1.),
                   #y_range=(1e-12, 1))

# Plot em
p.line(Rvals, hvals, legend='height', line_width=2)
#p.scatter(kdx_vals, error_fb, legend='fb', line_width=2, color='red')
#p.scatter(kdx_vals, error_sc, legend='sc', line_width=2, color='green')
#p.scatter(kdx_vals, error_4, legend='4', line_width=2, color='black')

p.legend.location = "top_left"

os_plotting.bokeh_set_fontsize(p, 16)

plotting.show(p)

Loading BokehJS ...

### To calculate the depth at the center $r=0$, we conserve mass (or, equivalently, volume):

### $V = \pi R^2 H$  -- Volume calculated before spinning

### Integrate our new shape from $r = 0$ to $R_{\rm tank}$ with a height $H(r=0) = H_{\rm new}$ 

### and $H(r) = H_{\rm new} + h(r)$

### with  $h(r) = \frac{1}{2} \frac{\Omega^2}{g} r^2$, as above

### we have $V = \int 2 \pi r H(r) dr = \pi R_{\rm tank}^2 H_{\rm new} + \frac{\pi}{4} \frac{\Omega^2}{g} R_{\rm tank}^4$

### Solving for $H_{\rm new}$, 

### $H_{\rm new} = (V - \frac{\pi}{4} \frac{\Omega^2}{g} R_{\rm tank}^4) / \pi R_{\rm tank}^2$

In [97]:
V = np.pi * R_tank**2 * H_water
V.to('cm**3')

unyt_quantity(14137.16694115, 'cm**3')

In [99]:
Hnew = (V - (np.pi/4) * (Omega_tank**2/g) * R_tank**4) / (np.pi * R_tank**2)
Hnew.to('cm')

unyt_quantity(19.43350485, 'cm')

### The center of the water has dropped approximately 0.5cm!

----

## <font color='blue'>PS 1-5</font> -- Merry-go round

### The major inconsistency of the merry-go round for describing the Coriolis force in isolation is that the balls will experience a significant Centrifugal force.  This will generally affect the trajectories resulting in motion distince from the Coriolis force in isolation.

### Dye drops place in a rotating tank at radius $r$ will enter the water with zero angular momenum (i.e. no rotatonal motion relative to the tank).  It will therefore travel towards the center, interact with the water via viscocity?, experience the Coriolis force and deflect accordingly.

----

## 2-4 Hockey  <font color='red'>(not assigned)</font> 

In [101]:
# Dartmouth
phi_Dartmouth = (43 + 38./60) * unyt.deg
D_inertial = 26 * unyt.m

### We desire inertial motion with diameter $D_{\rm inertial} = 26$m.

### As derived above, $R = \frac{V}{f}$ with $V$ the speed of the puck and $f$ calculated for Darthmouth.

### Rearranging:

### $V = f (D/2)$

In [104]:
f_Dartmouth = coriolis.coriolis_parameter(phi_Dartmouth)
V = f * (D_inertial/2.)
V.to('cm/s')

unyt_quantity(0.14841309, 'cm/s')

### Wow, that is slow!

----

## 2-5 Stone on the Bridge <font color='blue'>(PS 1-6)</font> 

### Setup

In [108]:
phi_bridge = 35 * unyt.deg # N
h_bridge = 300 * unyt.m
g = 9.8 * unyt.m / unyt.s**2

### Direction  -- Using Cushman-Roisin coordinate systems

### In our local, rotating frame, the rotation vector for the Earth is $\bar \Omega = \Omega \cos\phi \, \hat j + \Omega \sin\phi \, \hat k$.

### The Coriolis force is:  $-\bar 2 \Omega \times \bar u$ 

### A stone dropped "down" has velocity $\bar u = -u \, \hat k$ 

### Taking our cross-product and being careful with the right-hand rule, there is an acceleration in the $-\hat i$ direction.   ie.  West.

### To calculate the $\hat i$ deflection, we first calculate the time in air:

### $t = \sqrt{2h/g}$

In [109]:
t_drop = np.sqrt(2 * h_bridge / g)
t_drop.to('s')

unyt_quantity(7.82460796, 's')

### Our horizontal acceleration

### $\frac{du}{dt} = 2 \Omega w \cos\phi$ 

### and gravitational acceleration gives  $w = g t$

### implying $u = \int (du/dt) dt = \Omega g \cos \phi t^2$

### and last we integrate $u = dx/dt$ to get the horizontal displacement

### $x = \frac{1}{3} \Omega g \cos \phi t^3$  (all integral constants are zero as we started at rest in $x$)

In [111]:
# Numerics
x = (1/3) * unyt.Omega_earth * g * np.cos(phi_bridge) * t_drop**3
x.to('cm')

unyt_quantity(9.34783674, 'cm')

### Nearly 10cm !

----